# Action Recognition @ UCF101  
**Due date: 11:59 pm on Dec. 11, 2018 (Tuesday)**

## Description
---
In this homework, you will be doing action recognition using Recurrent Neural Network (RNN), (Long-Short Term Memory) LSTM in particular. You will be given a dataset called UCF101, which consists of 101 different actions/classes and for each action, there will be 145 samples. We tagged each sample into either training or testing. Each sample is supposed to be a short video, but we sampled 25 frames from each videos to reduce the data amount. Consequently, a training sample is a tuple of 3D volume with one dimension encoding *temporal correlation* between frames and a label indicating what action it is.

To tackle this problem, we aim to build a neural network that can not only capture spatial information of each frame but also temporal information between frames. Fortunately, you don't have to do this on your own. RNN — a type of neural network designed to deal with time-series data — is right here for you to use. In particular, you will be using LSTM for this task.

Instead of training a end-to-end neural network from scratch whose computation is prohibitively expensive for CPUs. We divide this into two steps: feature extraction and modelling. Below are the things you need to implement for this homework:
- **{35 pts} Feature extraction**. Use the pretrained VGG network to extract features from each frame. Specifically, we recommend  to use the activations of the first fully connected layer `torchvision.models.vgg16` (4096 dim) as features of each video frame. This will result into a 4096x25 matrix for each video. 
    **hints**: 
    - use `scipy.io.savemat()` to save feature to '.mat' file and `scipy.io.loadmat()` load feature.
    - norm your images using `torchvision.transforms`
    ```
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    prep = transforms.Compose([ transforms.ToTensor(), normalize ])
    prep(img)
    
    ```
    More detils of image preprocessing in PyTorch can be found at http://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    
- **{35 pts} Modelling**. With the extracted features, build an LSTM network which takes a 4096x25 sample as input, and outputs the action label of that sample.
- **{20 pts} Evaluation**. After training your network, you need to evaluate your model with the testing data by computing the prediction accuracy. Moreover, you need to compare the result of your network with that of support vector machine (SVM) (stacking the 4096x25 feature matrix to a long vector and train a SVM).
- **{10 pts} Report**. Details regarding the report can be found in the submission section below.

Notice that the size of the raw images is 256x340, whereas VGG16 takes 224x224 images as inputs. To solve this problem, instead of resizing the images which unfavorably changes the spatial ratio, we take a better solution: Cropping five 224x224 images at the image center and four corners and compute the 4096-dim VGG16 features for each of them, and average these five 4096-dim feature to get final feature representation for the raw image.

In order to save you computational time, we did the feature extraction of most samples for you except for class 1. For class 1, we provide you with the raw images, and you need to write code to extract the feature of the samples in class 1. Instead of training over the whole dataset on CPUs which mays cost you serval days, **use the first 15** classes of the whole dataset. The same applies to those who have access to GPUs.


## Dataset
Download dataset at [UCF101](http://vision.cs.stonybrook.edu/~yangwang/public/UCF101_dimitris_course.zip). 

The dataset is consist of the following two parts: video images and extracted features.

### 1. Video Images  

UCF101 dataset contains 101 actions and 13,320 videos in total.  

+ `annos/actions.txt`  
  + lists all the actions (`ApplyEyeMakeup`, .., `YoYo`)   
  
+ `annots/videos_labels_subsets.txt`  
  + lists all the videos (`v_000001`, .., `v_013320`)  
  + labels (`1`, .., `101`)  
  + subsets (`1` for train, `2` for test)  

+ `images_class1/`  
  + contains videos belonging to class 1 (`ApplyEyeMakeup`)  
  + each video folder contains 25 frames  


### 2. Video Features

+ `extract_vgg16_relu6.py`  
  + used to extract video features  
     + Given an image (size: 256x340), we get 5 crops (size: 224x224) at the image center and four corners. The `vgg16-relu6` features are extracted for all 5 crops and subsequently averaged to form a single feature vector (size: 4096).  
     + Given a video, we process its 25 images seuqentially. In the end, each video is represented as a feature sequence (size: 4096 x 25).  
  + written in PyTorch; supports both CPU and GPU.  

+ `vgg16_relu6/`  
   + contains all the video features, EXCEPT those belonging to class 1 (`ApplyEyeMakeup`)  
   + you need to run script `extract_vgg16_relu6.py` to complete the feature extracting process   


## Some Tutorials
- Good materials for understanding RNN and LSTM
    - http://blog.echen.me
    - http://karpathy.github.io/2015/05/21/rnn-effectiveness/
    - http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Implementing RNN and LSTM with PyTorch
    - [LSTM with PyTorch](http://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py)
    - [RNN with PyTorch](http://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)

# My Submission Contents:

1.	Ipynb with the best results of lstm and svm (Ray_Shayan_111424665_Report.ipynb)
2.	Vgg extraction – py file (Ray_Shayan_111424665_extract_vgg16_relu6.py)
3.	PDF report (Ray_Shayan_111424665_Report.pdf)
4.	Folder of 145 videos (.mat) for class 1 (Ray_Shayan_111424665_Class1_MAT145)


In [4]:
# write your codes here
import torch
import torch.nn as nn
import torch.utils.data as data_utils

from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import glob
import scipy.io
from PIL import Image
import torchvision.models as models
from torch.optim import lr_scheduler
import os
import numpy as np
import h5py

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as DD
import torchnet as tnt

import time

## ==========================================
##    Q1: FEATURE EXTRACTION for Images of Class 1 (also available in Ray_Shayan_111424665_extract_vgg16_relu6.py ) in this current folder
## ==========================================


In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
prep = transforms.Compose([ transforms.ToTensor(), normalize ])
#prep(img)

In [3]:

class1_path = './UCF101_release/images_class1/*'
class1_path_new = './UCF101_release/images_class1/'

vgg_videos = './UCF101_release/vgg16_relu6/'
frame_names = [name[-8:] for name in glob.glob(class1_path )]
image_class1_paths = [name for name in glob.glob(class1_path)]
print(len(frame_names))
#print(frame_names)
#print(image_class1_paths)

145


In [5]:
model_vgg16 = models.vgg16(pretrained=True) 
print("------------")
for param in model_vgg16.parameters():
    param.requires_grad = False

## outputs contain the 1st ReLU output of the classifier layer in the VGG 16 network as given in question
## we will outputs as vgg features to process the next steps in this problem
outputs= list()
def hook(module, input, output):
    outputs.append(output)

print(model_vgg16.classifier[1])
model_vgg16.classifier[1].register_forward_hook(hook)  #relu

print("------------")
print(model_vgg16.features)
print(model_vgg16.classifier)


------------
ReLU(inplace)
------------
Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kern

In [6]:
def get_vgg_feature(img):
    #features = vgg16_FC1(img)
    features = model_vgg16(img)
    return features
    

def load_crop_extract_save(aImgClass1Path):
    newImgTnsrs = dict()
    newImgTnsrs['feats'] = list()
    for image_filename in glob.glob(aImgClass1Path + '/*'):
        
        print("Processing image .... ",image_filename)
        # for each image crop 5 areas
        img = Image.open(image_filename)
        
        height = img.size[1]
        width = img.size[0]
        
        # tuple defining the left, upper, right, and lower pixel coordinates
        top_left = img.crop((0, 0, 224, 224))
        top_right = img.crop((width - 224, 0, width, 224))
        bottom_left = img.crop((0, height - 224, 224 , height))
        bottom_right = img.crop(( width - 224, height - 224, width, height))
        center =  img.crop(( (width/2) - 112, (height/2) - 112, (width/2) + 112, (height/2) + 112 ))
        
        #normalize and transform cropped images before passing to vgg 
        stcked_tnsr = torch.stack((prep(top_left), prep(top_right),prep(bottom_left) ,prep(bottom_right),prep(center)) , 0 )
        print('stcked_tnsr ',stcked_tnsr.shape)
        stcked_vgg = get_vgg_feature(stcked_tnsr)
        print(stcked_vgg.shape)
        print('output ',len(outputs))
        
        ## find mean
        new_img_mean = torch.mean(outputs[0], 0)
        print('new_img_mean ',new_img_mean.shape)
        newImgTnsrs['feats'].append(new_img_mean.numpy())
        outputs.clear() ## so that next image 
        
        #break ## comment out
    
    print('aImgTnsr >> ',len(newImgTnsrs['feats']))
    scipy.io.savemat(vgg_videos +aImgClass1Path[-8:], newImgTnsrs)
        
        
        
        
    

In [2]:
################################### UNCOMMENT THE CODE BELOW TO RUN EXTRACT VGG FEATURES (MAT FILES) ################## 
## get image features for class 1 - extract features - UNCOMMENT if you want to extract features again for class1
'''
start_time = time.time()
for aClass1Img in image_class1_paths: #glob.glob(image_class1_paths + '/*'):
    print("extracting .. ",aClass1Img)
    load_crop_extract_save(aClass1Img)
    
    #break  # comment out for all 
end_time = time.time() - start_time
print('------------------------------------------------------------')
print("time taken to extract features for Class 1  in seconds {:.3f}".format(end_time))
'''

'\nstart_time = time.time()\nfor aClass1Img in image_class1_paths: #glob.glob(image_class1_paths + \'/*\'):\n    print("extracting .. ",aClass1Img)\n    load_crop_extract_save(aClass1Img)\n    \n    #break  # comment out for all \nend_time = time.time() - start_time\nprint(\'------------------------------------------------------------\')\nprint("time taken to extract features for Class 1  in seconds {:.3f}".format(end_time))\n'

## Q2. Evaluation (LSTM)

In [10]:
# LSTM classification model
class LSTMClassify(nn.Module):
    def __init__(self):
        super(LSTMClassify, self).__init__()
        
        
        ## PREVIOUS ATTEMPTS ###
        '''
        self.recurrent_layer = nn.LSTM(input_size=4096, hidden_size=1200, num_layers=2, batch_first=True, bidirectional=True, dropout=0.25)
        self.classify_layer = nn.Linear(in_features=1200*2, out_features=15)

        self.recurrent_layer = nn.LSTM(input_size=4096, hidden_size=1200, num_layers=2, batch_first=True, bidirectional=True, dropout=0.25)
        self.classify_layer = nn.Linear(in_features=1200*2, out_features=15)
        
        self.recurrent_layer = nn.LSTM(input_size=4096, hidden_size=1200, num_layers=2, batch_first=True, dropout=0.25)
        self.classify_layer = nn.Linear(in_features=1200, out_features=15) 
       
        '''
        ############## Network Architecture ##############
         ###################################################
        ## FINAL ::changed num hidden layers to 1500
        self.recurrent_layer = nn.LSTM(input_size=4096, hidden_size=1500, num_layers=2, batch_first=True, bidirectional=True, dropout=0.25)
        self.classify_layer = nn.Linear(in_features=1500*2, out_features=15) ##  question talks about taking only first 15 classes
        ###################################################
    
    # the size of input is [batch_size, seq_len(25), input_dim(4096)]
    # the size of logits is [batch_size, num_class]
    
    def forward(self, input, h_t_1=None, c_t_1=None):
        
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        rnn_outputs, (hn, cn) = self.recurrent_layer(input)
        
        # classify the last step of rnn_outpus
        # the size of logits is [batch_size, num_class]
        logits = self.classify_layer(rnn_outputs[:,-1])
        return logits

model_lstm = LSTMClassify() 


#### optimizers and loss functions for custom classifier
#optimizer = torch.optim.Adam(params=model_lstm.parameters(), lr=3e-3, betas=(0.1, 0.2))
optimizer = torch.optim.SGD(model_lstm.parameters(), lr = 0.0002, momentum=0.95) ## try SGD
criterion = nn.CrossEntropyLoss()
num_class = 15

In [11]:

train_features = list()
test_features = list()

train_lbls = list()
test_lbls = list()

mat_file_path = './UCF101_release/vgg16_relu6/'
vls_path = './UCF101_release/annos/videos_labels_subsets.txt'

with open(vls_path,'r') as lblfile:
    for line in lblfile:
        line_arr = line.strip().split('\t') 
    
        video = line_arr[0]
        lbl = line_arr[1]
        trn_test = line_arr[2] # 1 = train; 2 = test
        
        print("Processing ... Video %s Label %s Train/Test %s"%(video, lbl, trn_test))
        #print("Loading file .... ", (mat_file_path + video))

        if int(lbl) > 15:
            break  ## only the first 15 classes to be classified
        
        if int(trn_test) == 1: # 1 = train;
            trn_mat = scipy.io.loadmat(mat_file_path + video ) #+ '.mat'
            if int(lbl) == 1:
                trn_feature = np.array(trn_mat['feats'])#.flatten()
            else:
                trn_feature = np.array(trn_mat['Feature'])#.flatten()
            train_features.append(trn_feature)
            train_lbls.append(int(lbl) -  1)
        else:
            test_mat = scipy.io.loadmat(mat_file_path + video ) #+ '.mat'
            #print(test_mat['feats'])
            if int(lbl) == 1:
                test_feature = np.array(test_mat['feats'])#.flatten()
            else:
                test_feature = np.array(test_mat['Feature'])#.flatten()
            test_features.append(test_feature)
            test_lbls.append(int(lbl) -  1)

# tried batch_size = 5, 15
train = DD.TensorDataset(torch.Tensor(np.asarray(train_features)), torch.Tensor(np.asarray(train_lbls)))
train_loader = DD.DataLoader(train, batch_size=15, shuffle=False, sampler=DD.sampler.RandomSampler(train),num_workers=2, pin_memory=False)

Processing ... Video v_000001 Label 1 Train/Test 2
Processing ... Video v_000002 Label 1 Train/Test 2
Processing ... Video v_000003 Label 1 Train/Test 2
Processing ... Video v_000004 Label 1 Train/Test 2
Processing ... Video v_000005 Label 1 Train/Test 2
Processing ... Video v_000006 Label 1 Train/Test 2
Processing ... Video v_000007 Label 1 Train/Test 2
Processing ... Video v_000008 Label 1 Train/Test 2
Processing ... Video v_000009 Label 1 Train/Test 2
Processing ... Video v_000010 Label 1 Train/Test 2
Processing ... Video v_000011 Label 1 Train/Test 2
Processing ... Video v_000012 Label 1 Train/Test 2
Processing ... Video v_000013 Label 1 Train/Test 2
Processing ... Video v_000014 Label 1 Train/Test 2
Processing ... Video v_000015 Label 1 Train/Test 2
Processing ... Video v_000016 Label 1 Train/Test 2
Processing ... Video v_000017 Label 1 Train/Test 2
Processing ... Video v_000018 Label 1 Train/Test 2
Processing ... Video v_000019 Label 1 Train/Test 2
Processing ... Video v_000020 L

Processing ... Video v_000205 Label 2 Train/Test 1
Processing ... Video v_000206 Label 2 Train/Test 1
Processing ... Video v_000207 Label 2 Train/Test 1
Processing ... Video v_000208 Label 2 Train/Test 1
Processing ... Video v_000209 Label 2 Train/Test 1
Processing ... Video v_000210 Label 2 Train/Test 1
Processing ... Video v_000211 Label 2 Train/Test 1
Processing ... Video v_000212 Label 2 Train/Test 1
Processing ... Video v_000213 Label 2 Train/Test 1
Processing ... Video v_000214 Label 2 Train/Test 1
Processing ... Video v_000215 Label 2 Train/Test 1
Processing ... Video v_000216 Label 2 Train/Test 1
Processing ... Video v_000217 Label 2 Train/Test 1
Processing ... Video v_000218 Label 2 Train/Test 1
Processing ... Video v_000219 Label 2 Train/Test 1
Processing ... Video v_000220 Label 2 Train/Test 1
Processing ... Video v_000221 Label 2 Train/Test 1
Processing ... Video v_000222 Label 2 Train/Test 1
Processing ... Video v_000223 Label 2 Train/Test 1
Processing ... Video v_000224 L

Processing ... Video v_000414 Label 4 Train/Test 2
Processing ... Video v_000415 Label 4 Train/Test 2
Processing ... Video v_000416 Label 4 Train/Test 2
Processing ... Video v_000417 Label 4 Train/Test 2
Processing ... Video v_000418 Label 4 Train/Test 2
Processing ... Video v_000419 Label 4 Train/Test 2
Processing ... Video v_000420 Label 4 Train/Test 2
Processing ... Video v_000421 Label 4 Train/Test 2
Processing ... Video v_000422 Label 4 Train/Test 2
Processing ... Video v_000423 Label 4 Train/Test 2
Processing ... Video v_000424 Label 4 Train/Test 2
Processing ... Video v_000425 Label 4 Train/Test 2
Processing ... Video v_000426 Label 4 Train/Test 2
Processing ... Video v_000427 Label 4 Train/Test 2
Processing ... Video v_000428 Label 4 Train/Test 2
Processing ... Video v_000429 Label 4 Train/Test 2
Processing ... Video v_000430 Label 4 Train/Test 2
Processing ... Video v_000431 Label 4 Train/Test 2
Processing ... Video v_000432 Label 4 Train/Test 2
Processing ... Video v_000433 L

Processing ... Video v_000613 Label 5 Train/Test 1
Processing ... Video v_000614 Label 5 Train/Test 1
Processing ... Video v_000615 Label 5 Train/Test 1
Processing ... Video v_000616 Label 5 Train/Test 1
Processing ... Video v_000617 Label 5 Train/Test 1
Processing ... Video v_000618 Label 5 Train/Test 1
Processing ... Video v_000619 Label 5 Train/Test 1
Processing ... Video v_000620 Label 5 Train/Test 1
Processing ... Video v_000621 Label 5 Train/Test 1
Processing ... Video v_000622 Label 5 Train/Test 1
Processing ... Video v_000623 Label 5 Train/Test 1
Processing ... Video v_000624 Label 5 Train/Test 1
Processing ... Video v_000625 Label 5 Train/Test 1
Processing ... Video v_000626 Label 5 Train/Test 1
Processing ... Video v_000627 Label 5 Train/Test 1
Processing ... Video v_000628 Label 5 Train/Test 1
Processing ... Video v_000629 Label 5 Train/Test 1
Processing ... Video v_000630 Label 5 Train/Test 1
Processing ... Video v_000631 Label 5 Train/Test 1
Processing ... Video v_000632 L

Processing ... Video v_000845 Label 7 Train/Test 1
Processing ... Video v_000846 Label 7 Train/Test 1
Processing ... Video v_000847 Label 7 Train/Test 1
Processing ... Video v_000848 Label 7 Train/Test 1
Processing ... Video v_000849 Label 7 Train/Test 1
Processing ... Video v_000850 Label 7 Train/Test 1
Processing ... Video v_000851 Label 7 Train/Test 1
Processing ... Video v_000852 Label 7 Train/Test 1
Processing ... Video v_000853 Label 7 Train/Test 1
Processing ... Video v_000854 Label 7 Train/Test 1
Processing ... Video v_000855 Label 7 Train/Test 1
Processing ... Video v_000856 Label 7 Train/Test 1
Processing ... Video v_000857 Label 7 Train/Test 1
Processing ... Video v_000858 Label 7 Train/Test 1
Processing ... Video v_000859 Label 7 Train/Test 1
Processing ... Video v_000860 Label 7 Train/Test 1
Processing ... Video v_000861 Label 7 Train/Test 1
Processing ... Video v_000862 Label 7 Train/Test 1
Processing ... Video v_000863 Label 7 Train/Test 1
Processing ... Video v_000864 L

Processing ... Video v_001043 Label 8 Train/Test 1
Processing ... Video v_001044 Label 8 Train/Test 1
Processing ... Video v_001045 Label 8 Train/Test 1
Processing ... Video v_001046 Label 8 Train/Test 1
Processing ... Video v_001047 Label 8 Train/Test 1
Processing ... Video v_001048 Label 8 Train/Test 1
Processing ... Video v_001049 Label 8 Train/Test 1
Processing ... Video v_001050 Label 8 Train/Test 1
Processing ... Video v_001051 Label 8 Train/Test 1
Processing ... Video v_001052 Label 8 Train/Test 1
Processing ... Video v_001053 Label 8 Train/Test 1
Processing ... Video v_001054 Label 8 Train/Test 1
Processing ... Video v_001055 Label 8 Train/Test 1
Processing ... Video v_001056 Label 8 Train/Test 1
Processing ... Video v_001057 Label 8 Train/Test 1
Processing ... Video v_001058 Label 8 Train/Test 1
Processing ... Video v_001059 Label 8 Train/Test 1
Processing ... Video v_001060 Label 8 Train/Test 1
Processing ... Video v_001061 Label 8 Train/Test 1
Processing ... Video v_001062 L

Processing ... Video v_001260 Label 10 Train/Test 2
Processing ... Video v_001261 Label 10 Train/Test 2
Processing ... Video v_001262 Label 10 Train/Test 2
Processing ... Video v_001263 Label 10 Train/Test 1
Processing ... Video v_001264 Label 10 Train/Test 1
Processing ... Video v_001265 Label 10 Train/Test 1
Processing ... Video v_001266 Label 10 Train/Test 1
Processing ... Video v_001267 Label 10 Train/Test 1
Processing ... Video v_001268 Label 10 Train/Test 1
Processing ... Video v_001269 Label 10 Train/Test 1
Processing ... Video v_001270 Label 10 Train/Test 1
Processing ... Video v_001271 Label 10 Train/Test 1
Processing ... Video v_001272 Label 10 Train/Test 1
Processing ... Video v_001273 Label 10 Train/Test 1
Processing ... Video v_001274 Label 10 Train/Test 1
Processing ... Video v_001275 Label 10 Train/Test 1
Processing ... Video v_001276 Label 10 Train/Test 1
Processing ... Video v_001277 Label 10 Train/Test 1
Processing ... Video v_001278 Label 10 Train/Test 1
Processing .

Processing ... Video v_001493 Label 11 Train/Test 1
Processing ... Video v_001494 Label 11 Train/Test 1
Processing ... Video v_001495 Label 11 Train/Test 1
Processing ... Video v_001496 Label 11 Train/Test 1
Processing ... Video v_001497 Label 11 Train/Test 1
Processing ... Video v_001498 Label 11 Train/Test 1
Processing ... Video v_001499 Label 11 Train/Test 1
Processing ... Video v_001500 Label 11 Train/Test 1
Processing ... Video v_001501 Label 11 Train/Test 1
Processing ... Video v_001502 Label 11 Train/Test 1
Processing ... Video v_001503 Label 11 Train/Test 1
Processing ... Video v_001504 Label 11 Train/Test 1
Processing ... Video v_001505 Label 11 Train/Test 1
Processing ... Video v_001506 Label 11 Train/Test 1
Processing ... Video v_001507 Label 11 Train/Test 1
Processing ... Video v_001508 Label 11 Train/Test 1
Processing ... Video v_001509 Label 12 Train/Test 2
Processing ... Video v_001510 Label 12 Train/Test 2
Processing ... Video v_001511 Label 12 Train/Test 2
Processing .

Processing ... Video v_001687 Label 13 Train/Test 2
Processing ... Video v_001688 Label 13 Train/Test 2
Processing ... Video v_001689 Label 13 Train/Test 2
Processing ... Video v_001690 Label 13 Train/Test 2
Processing ... Video v_001691 Label 13 Train/Test 2
Processing ... Video v_001692 Label 13 Train/Test 2
Processing ... Video v_001693 Label 13 Train/Test 2
Processing ... Video v_001694 Label 13 Train/Test 2
Processing ... Video v_001695 Label 13 Train/Test 2
Processing ... Video v_001696 Label 13 Train/Test 2
Processing ... Video v_001697 Label 13 Train/Test 1
Processing ... Video v_001698 Label 13 Train/Test 1
Processing ... Video v_001699 Label 13 Train/Test 1
Processing ... Video v_001700 Label 13 Train/Test 1
Processing ... Video v_001701 Label 13 Train/Test 1
Processing ... Video v_001702 Label 13 Train/Test 1
Processing ... Video v_001703 Label 13 Train/Test 1
Processing ... Video v_001704 Label 13 Train/Test 1
Processing ... Video v_001705 Label 13 Train/Test 1
Processing .

Processing ... Video v_001890 Label 14 Train/Test 1
Processing ... Video v_001891 Label 14 Train/Test 1
Processing ... Video v_001892 Label 14 Train/Test 1
Processing ... Video v_001893 Label 14 Train/Test 1
Processing ... Video v_001894 Label 14 Train/Test 1
Processing ... Video v_001895 Label 14 Train/Test 1
Processing ... Video v_001896 Label 14 Train/Test 1
Processing ... Video v_001897 Label 14 Train/Test 1
Processing ... Video v_001898 Label 14 Train/Test 1
Processing ... Video v_001899 Label 15 Train/Test 2
Processing ... Video v_001900 Label 15 Train/Test 2
Processing ... Video v_001901 Label 15 Train/Test 2
Processing ... Video v_001902 Label 15 Train/Test 2
Processing ... Video v_001903 Label 15 Train/Test 2
Processing ... Video v_001904 Label 15 Train/Test 2
Processing ... Video v_001905 Label 15 Train/Test 2
Processing ... Video v_001906 Label 15 Train/Test 2
Processing ... Video v_001907 Label 15 Train/Test 2
Processing ... Video v_001908 Label 15 Train/Test 2
Processing .

In [12]:
#./UCF101_release/vgg16_relu6/v_000146
# test load and view contents
try1 = scipy.io.loadmat('./UCF101_release/vgg16_relu6/v_000146') #+ '.mat'
print(try1.items())
print(try1['Feature'])
        

dict_items([('__header__', b'MATLAB 5.0 MAT-file Platform: posix, Created on: Wed Nov  8 11:45:03 2017'), ('__version__', '1.0'), ('__globals__', []), ('Feature', array([[0.2503409 , 1.2001994 , 1.4392091 , ..., 0.03667732, 0.26077977,
        2.531755  ],
       [0.20965204, 0.7805546 , 1.787047  , ..., 0.        , 0.45286703,
        2.018227  ],
       [0.37332922, 0.8460091 , 1.3158264 , ..., 0.        , 0.6594631 ,
        1.4026463 ],
       ...,
       [0.6048986 , 0.576588  , 1.811272  , ..., 0.        , 0.96530473,
        2.400215  ],
       [0.545093  , 0.5584324 , 1.8760223 , ..., 0.07922463, 0.8918521 ,
        2.1957998 ],
       [0.4909359 , 0.27355734, 1.6488619 , ..., 0.        , 1.3855635 ,
        2.0995514 ]], dtype=float32))])
[[0.2503409  1.2001994  1.4392091  ... 0.03667732 0.26077977 2.531755  ]
 [0.20965204 0.7805546  1.787047   ... 0.         0.45286703 2.018227  ]
 [0.37332922 0.8460091  1.3158264  ... 0.         0.6594631  1.4026463 ]
 ...
 [0.6048986  0.576

In [13]:
use_cuda = torch.cuda.is_available()
print('use cuda: %s'%(use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor

use cuda: False


In [14]:
# run the model for one epoch
# can be used for both training or validation model
def run_epoch(data_loader, model, criterion, epoch, is_training, optimizer=None):
    
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()
    
    #print(enumerate(data_loader))
    for batch_idx, sample in enumerate(data_loader):
        #print(batch_idx , sample)
        #sequence = sample['seq']
        #label = sample['label']
        input_sequence_var = sample[0] #Variable(sequence).type(FloatTensor)
        input_label_var =  Variable(sample[1]).type(LongTensor)
        

        # compute output
        # output_logits: [batch_size, num_class]
        #print("input_sequence_var >> ",input_sequence_var.shape)
        output_logits = model(input_sequence_var)
        loss = criterion(output_logits, input_label_var)
        
        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.data[0])
        print("loss >>> ",loss.data[0])
        acc.add(output_logits.data, input_label_var.data)
        confusion_matrix.add(output_logits.data, input_label_var.data)


    print('%s Epoch: %d  , Loss: %.4f,  Accuracy: %.2f'%(logger_prefix, epoch, meter_loss.value()[0], acc.value()[0]))
    return acc.value()[0]

start_time = time.time()
num_epochs = 20
evaluate_every_epoch = 5
for e in range(num_epochs):
    run_epoch(train_loader, model_lstm, criterion, e, True, optimizer)
    #if e % evaluate_every_epoch == 0:
    #    run_epoch(valLD, model_lstm, criterion, e, False, None)   

end_time = time.time() - start_time
print('------------------------------------------------------------')
print("time taken to TRAIN for LSTM Model  in seconds {:.3f}".format(end_time))

/home/shayan_ray/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/shayan_ray/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


loss >>>  tensor(2.7050)
loss >>>  tensor(2.7152)
loss >>>  tensor(2.7002)
loss >>>  tensor(2.7101)
loss >>>  tensor(2.6905)
loss >>>  tensor(2.6946)
loss >>>  tensor(2.7058)
loss >>>  tensor(2.6953)
loss >>>  tensor(2.7061)
loss >>>  tensor(2.7180)
loss >>>  tensor(2.7181)
loss >>>  tensor(2.7183)
loss >>>  tensor(2.6988)
loss >>>  tensor(2.6942)
loss >>>  tensor(2.6916)
loss >>>  tensor(2.6963)
loss >>>  tensor(2.7047)
loss >>>  tensor(2.6799)
loss >>>  tensor(2.6818)
loss >>>  tensor(2.6700)
loss >>>  tensor(2.6969)
loss >>>  tensor(2.6909)
loss >>>  tensor(2.6911)
loss >>>  tensor(2.6789)
loss >>>  tensor(2.6903)
loss >>>  tensor(2.6895)
loss >>>  tensor(2.6467)
loss >>>  tensor(2.6714)
loss >>>  tensor(2.6640)
loss >>>  tensor(2.6561)
loss >>>  tensor(2.6737)
loss >>>  tensor(2.6876)
loss >>>  tensor(2.6652)
loss >>>  tensor(2.6650)
loss >>>  tensor(2.6791)
loss >>>  tensor(2.6625)
loss >>>  tensor(2.6650)
loss >>>  tensor(2.6543)
loss >>>  tensor(2.6723)
loss >>>  tensor(2.6933)


loss >>>  tensor(2.2230)
loss >>>  tensor(2.2792)
loss >>>  tensor(2.2907)
loss >>>  tensor(2.3175)
loss >>>  tensor(2.0104)
loss >>>  tensor(2.1344)
loss >>>  tensor(2.0356)
loss >>>  tensor(2.3105)
loss >>>  tensor(2.0166)
loss >>>  tensor(2.0910)
loss >>>  tensor(2.1153)
loss >>>  tensor(2.2671)
loss >>>  tensor(2.0603)
loss >>>  tensor(2.2442)
loss >>>  tensor(2.1167)
loss >>>  tensor(2.0089)
loss >>>  tensor(2.2729)
loss >>>  tensor(1.7475)
loss >>>  tensor(1.9091)
loss >>>  tensor(2.2626)
loss >>>  tensor(2.2236)
loss >>>  tensor(2.0765)
loss >>>  tensor(1.9097)
loss >>>  tensor(2.2175)
loss >>>  tensor(1.8566)
loss >>>  tensor(2.1976)
loss >>>  tensor(2.0310)
loss >>>  tensor(2.1005)
loss >>>  tensor(2.0602)
loss >>>  tensor(2.1375)
loss >>>  tensor(2.0916)
loss >>>  tensor(1.8518)
loss >>>  tensor(1.9422)
loss >>>  tensor(2.0968)
loss >>>  tensor(2.1842)
loss >>>  tensor(2.1759)
loss >>>  tensor(1.8542)
loss >>>  tensor(2.2165)
loss >>>  tensor(1.8578)
loss >>>  tensor(2.2273)


loss >>>  tensor(1.3796)
loss >>>  tensor(1.4403)
loss >>>  tensor(1.5120)
loss >>>  tensor(1.2921)
loss >>>  tensor(1.1286)
loss >>>  tensor(0.9995)
loss >>>  tensor(1.2326)
loss >>>  tensor(1.4144)
loss >>>  tensor(1.4688)
loss >>>  tensor(1.2669)
loss >>>  tensor(1.0804)
loss >>>  tensor(1.3095)
loss >>>  tensor(1.1024)
loss >>>  tensor(1.1543)
loss >>>  tensor(1.2878)
loss >>>  tensor(1.3721)
loss >>>  tensor(1.4173)
loss >>>  tensor(1.6504)
loss >>>  tensor(1.2430)
loss >>>  tensor(1.2627)
loss >>>  tensor(1.4370)
loss >>>  tensor(1.4158)
loss >>>  tensor(1.2639)
loss >>>  tensor(1.2856)
loss >>>  tensor(1.4719)
loss >>>  tensor(0.8429)
train Epoch: 6  , Loss: 1.4101,  Accuracy: 79.82
loss >>>  tensor(0.8683)
loss >>>  tensor(1.5121)
loss >>>  tensor(1.4097)
loss >>>  tensor(1.2590)
loss >>>  tensor(1.3776)
loss >>>  tensor(1.3760)
loss >>>  tensor(1.3706)
loss >>>  tensor(1.1080)
loss >>>  tensor(1.0840)
loss >>>  tensor(1.2902)
loss >>>  tensor(1.2762)
loss >>>  tensor(1.2762)
l

loss >>>  tensor(0.4746)
loss >>>  tensor(0.5824)
loss >>>  tensor(0.7791)
loss >>>  tensor(0.5764)
loss >>>  tensor(0.9847)
loss >>>  tensor(0.6675)
loss >>>  tensor(0.6985)
loss >>>  tensor(0.6148)
loss >>>  tensor(0.7870)
loss >>>  tensor(0.7086)
loss >>>  tensor(0.6870)
loss >>>  tensor(0.5824)
loss >>>  tensor(0.7566)
loss >>>  tensor(0.5470)
loss >>>  tensor(0.7803)
loss >>>  tensor(0.7235)
loss >>>  tensor(0.7580)
loss >>>  tensor(0.7994)
loss >>>  tensor(0.8030)
loss >>>  tensor(0.8735)
loss >>>  tensor(0.7402)
loss >>>  tensor(0.7696)
loss >>>  tensor(0.7693)
loss >>>  tensor(0.6879)
loss >>>  tensor(0.4519)
loss >>>  tensor(0.7476)
loss >>>  tensor(0.6183)
loss >>>  tensor(0.6332)
loss >>>  tensor(0.5919)
loss >>>  tensor(0.7901)
loss >>>  tensor(0.8918)
loss >>>  tensor(0.8147)
loss >>>  tensor(0.8237)
loss >>>  tensor(0.5946)
loss >>>  tensor(0.5507)
loss >>>  tensor(0.8363)
loss >>>  tensor(0.6053)
loss >>>  tensor(0.6746)
loss >>>  tensor(0.7808)
loss >>>  tensor(0.6041)


loss >>>  tensor(0.2507)
loss >>>  tensor(0.3976)
loss >>>  tensor(0.2807)
loss >>>  tensor(0.2947)
loss >>>  tensor(0.5325)
loss >>>  tensor(0.3207)
loss >>>  tensor(0.3704)
loss >>>  tensor(0.3526)
loss >>>  tensor(0.5171)
loss >>>  tensor(0.4511)
loss >>>  tensor(0.4518)
loss >>>  tensor(0.2786)
loss >>>  tensor(0.3148)
loss >>>  tensor(0.2660)
loss >>>  tensor(0.3355)
loss >>>  tensor(0.3545)
loss >>>  tensor(0.5165)
loss >>>  tensor(0.4235)
loss >>>  tensor(0.3598)
loss >>>  tensor(0.2085)
loss >>>  tensor(0.3070)
loss >>>  tensor(0.4403)
loss >>>  tensor(0.4358)
loss >>>  tensor(0.3417)
loss >>>  tensor(0.3638)
loss >>>  tensor(0.3389)
loss >>>  tensor(0.3726)
loss >>>  tensor(0.2618)
loss >>>  tensor(0.5358)
loss >>>  tensor(0.3844)
loss >>>  tensor(0.3069)
loss >>>  tensor(0.7488)
loss >>>  tensor(0.4263)
loss >>>  tensor(0.4135)
loss >>>  tensor(0.2811)
loss >>>  tensor(0.3905)
loss >>>  tensor(0.3017)
loss >>>  tensor(0.3115)
loss >>>  tensor(0.2425)
loss >>>  tensor(0.3281)


loss >>>  tensor(0.1440)
loss >>>  tensor(0.1897)
loss >>>  tensor(0.3051)
loss >>>  tensor(0.1531)
loss >>>  tensor(0.1917)
loss >>>  tensor(0.1364)
loss >>>  tensor(0.1654)
loss >>>  tensor(0.1239)
loss >>>  tensor(0.2844)
loss >>>  tensor(0.2466)
loss >>>  tensor(0.1256)
loss >>>  tensor(0.2806)
loss >>>  tensor(0.1612)
loss >>>  tensor(0.1339)
loss >>>  tensor(0.1094)
loss >>>  tensor(0.1048)
loss >>>  tensor(0.1272)
loss >>>  tensor(0.1279)
loss >>>  tensor(0.1977)
loss >>>  tensor(0.1698)
loss >>>  tensor(0.2328)
loss >>>  tensor(0.1269)
loss >>>  tensor(0.1603)
loss >>>  tensor(0.1499)
loss >>>  tensor(0.1634)
loss >>>  tensor(0.2880)
loss >>>  tensor(0.1233)
loss >>>  tensor(0.3066)
loss >>>  tensor(0.2423)
loss >>>  tensor(0.1430)
loss >>>  tensor(0.1647)
loss >>>  tensor(0.2210)
train Epoch: 16  , Loss: 0.1927,  Accuracy: 98.61
loss >>>  tensor(0.0981)
loss >>>  tensor(0.1840)
loss >>>  tensor(0.1550)
loss >>>  tensor(0.2333)
loss >>>  tensor(0.1353)
loss >>>  tensor(0.1383)


loss >>>  tensor(0.0501)
train Epoch: 19  , Loss: 0.1047,  Accuracy: 99.72
------------------------------------------------------------
time taken to TRAIN for LSTM Model  in seconds 10453.741


In [15]:

testds = DD.TensorDataset(torch.Tensor(np.asarray(test_features)), torch.Tensor(np.asarray(test_lbls)))
test_loader = DD.DataLoader(testds, batch_size=5, shuffle=False, sampler=DD.sampler.SequentialSampler(testds),num_workers=2, pin_memory=False)

In [16]:
def predict_on_test(model_lstm, data_loader):
    model_lstm.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    count=0
    #confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    #meter_loss = tnt.meter.AverageValueMeter()
    
    for batch_idx, sample in enumerate(data_loader):
        input_sequence_var = sample[0] #Variable(sequence).type(FloatTensor)
        actual_label_var =  Variable(sample[1]).type(LongTensor)
        scores = model_lstm(input_sequence_var)
        print(torch.max(scores,1))
        _, preds = torch.max(scores,1) #scores.data.max(1)
        #print("preds %s  ",preds)
        #print("actual_label_var ",actual_label_var)
        acc.add(scores.data, actual_label_var.data)
    return acc.value()[0]

start_time = time.time()
accuracy=predict_on_test(model_lstm, test_loader)
print('Test Accuracy using custom LSTM model on this dataset is: ',accuracy)
end_time = time.time() - start_time 
print('------------------------------------------------------------')
print("time taken to Predict TEST data  in seconds {:.3f}".format(end_time))

(tensor([3.5518, 4.0968, 6.0707, 5.4259, 5.6939], grad_fn=<MaxBackward0>), tensor([12, 12,  1,  1,  1]))
(tensor([5.9195, 4.8216, 4.6072, 4.7648, 3.7450], grad_fn=<MaxBackward0>), tensor([ 1,  0,  1,  0, 13]))
(tensor([6.5793, 6.6177, 6.5000, 6.4033, 6.7030], grad_fn=<MaxBackward0>), tensor([1, 1, 1, 1, 1]))
(tensor([6.1853, 6.6734, 6.0828, 3.9354, 6.3354], grad_fn=<MaxBackward0>), tensor([ 1,  0,  0, 12,  0]))
(tensor([5.5898, 5.9734, 6.3852, 6.9862, 7.0849], grad_fn=<MaxBackward0>), tensor([0, 0, 0, 0, 0]))
(tensor([7.0522, 7.2217, 6.0295, 6.8838, 4.9997], grad_fn=<MaxBackward0>), tensor([0, 0, 0, 0, 1]))
(tensor([5.3167, 5.2274, 5.7813, 5.1736, 5.7453], grad_fn=<MaxBackward0>), tensor([1, 1, 1, 1, 1]))
(tensor([5.7434, 5.9274, 4.7967, 5.8844, 7.3177], grad_fn=<MaxBackward0>), tensor([ 1,  1, 12,  0,  0]))
(tensor([7.2100, 6.4100, 6.5018, 6.3698, 6.7556], grad_fn=<MaxBackward0>), tensor([0, 0, 0, 0, 1]))
(tensor([6.0451, 6.7590, 7.1970, 6.5808, 5.9417], grad_fn=<MaxBackward0>), tenso

(tensor([7.5007, 6.3488, 6.9138, 6.6674, 6.8217], grad_fn=<MaxBackward0>), tensor([10, 10, 10, 10, 10]))
(tensor([7.4589, 6.4079, 6.5933, 6.6940, 3.8814], grad_fn=<MaxBackward0>), tensor([10, 10, 10, 10, 10]))
(tensor([4.4747, 6.1881, 5.7125, 6.2618, 6.0800], grad_fn=<MaxBackward0>), tensor([10, 10, 10, 10, 10]))
(tensor([3.5680, 2.9862, 7.7805, 8.2021, 7.8710], grad_fn=<MaxBackward0>), tensor([10, 10, 11, 11, 11]))
(tensor([7.1929, 8.0400, 7.7878, 7.6354, 6.8926], grad_fn=<MaxBackward0>), tensor([11, 11, 11, 11, 11]))
(tensor([6.7068, 7.0463, 5.3286, 7.6354, 6.7068], grad_fn=<MaxBackward0>), tensor([11, 11, 11, 11, 11]))
(tensor([7.6375, 6.3868, 6.7269, 7.3964, 7.4907], grad_fn=<MaxBackward0>), tensor([11, 11, 11, 11, 11]))
(tensor([6.4339, 8.1205, 7.6291, 7.7915, 7.8023], grad_fn=<MaxBackward0>), tensor([11, 11, 11, 11, 11]))
(tensor([7.4669, 8.1094, 6.8852, 6.4780, 7.0551], grad_fn=<MaxBackward0>), tensor([11, 11, 11, 11, 11]))
(tensor([7.3353, 6.4518, 7.1718, 7.8700, 7.6173], grad_

## Q2. Evaluation (SVM)

In [5]:
#### SVM Training 
train_features_svm = list()
test_features_svm = list()

train_lbls_svm = list()
test_lbls_svm = list()

mat_file_path = './UCF101_release/vgg16_relu6/'
vls_path = './UCF101_release/annos/videos_labels_subsets.txt'

with open(vls_path,'r') as lblfile:
    for line in lblfile:
        line_arr = line.strip().split('\t') 
    
        video = line_arr[0]
        lbl = line_arr[1]
        trn_test = line_arr[2] # 1 = train; 2 = test
        
        print("Processing ... Video %s Label %s Train/Test %s"%(video, lbl, trn_test))
        #print("Loading file .... ", (mat_file_path + video))

        if int(lbl) > 15:
            break  ## only the first 15 classes to be classified
        
        if int(trn_test) == 1: # 1 = train;
            trn_mat = scipy.io.loadmat(mat_file_path + video ) #+ '.mat'
            if int(lbl) == 1:
                trn_feature = np.array(trn_mat['feats']).flatten()
            else:
                trn_feature = np.array(trn_mat['Feature']).flatten()
            train_features_svm.append(trn_feature)
            train_lbls_svm.append(int(lbl) -  1)
        else:
            test_mat = scipy.io.loadmat(mat_file_path + video ) #+ '.mat'
            #print(test_mat['feats'])
            if int(lbl) == 1:
                test_feature = np.array(test_mat['feats']).flatten()
            else:
                test_feature = np.array(test_mat['Feature']).flatten()
            test_features_svm.append(test_feature)
            test_lbls_svm.append(int(lbl) -  1)

### SVM
from sklearn import svm


## PREVIOUS ATTEMPTS ###
'''
model_svm = svm.LinearSVC(C=0.091, tol=5e-3, loss='hinge', penalty='l2', random_state=42)
model_svm = svm.SVC(kernel='poly', C=0.75, tol=5e-3)
model_svm = svm.LinearSVC(C=5)
model_svm = svm.SVC(C=0.091, tol=5e-3, kernel='rbf')
model_svm = svm.SVC(C=50)
'''
start_time = time.time()
model_svm = svm.SVC(C=1.0)
model_svm.fit(np.array(train_features_svm), np.array(train_lbls_svm))
end_time = time.time() - start_time 
print('------------------------------------------------------------')
print("time taken to TRAIN SVM  in seconds {:.3f}".format(end_time))

Processing ... Video v_000001 Label 1 Train/Test 2
Processing ... Video v_000002 Label 1 Train/Test 2
Processing ... Video v_000003 Label 1 Train/Test 2
Processing ... Video v_000004 Label 1 Train/Test 2
Processing ... Video v_000005 Label 1 Train/Test 2
Processing ... Video v_000006 Label 1 Train/Test 2
Processing ... Video v_000007 Label 1 Train/Test 2
Processing ... Video v_000008 Label 1 Train/Test 2
Processing ... Video v_000009 Label 1 Train/Test 2
Processing ... Video v_000010 Label 1 Train/Test 2
Processing ... Video v_000011 Label 1 Train/Test 2
Processing ... Video v_000012 Label 1 Train/Test 2
Processing ... Video v_000013 Label 1 Train/Test 2
Processing ... Video v_000014 Label 1 Train/Test 2
Processing ... Video v_000015 Label 1 Train/Test 2
Processing ... Video v_000016 Label 1 Train/Test 2
Processing ... Video v_000017 Label 1 Train/Test 2
Processing ... Video v_000018 Label 1 Train/Test 2
Processing ... Video v_000019 Label 1 Train/Test 2
Processing ... Video v_000020 L

Processing ... Video v_000324 Label 3 Train/Test 1
Processing ... Video v_000325 Label 3 Train/Test 1
Processing ... Video v_000326 Label 3 Train/Test 1
Processing ... Video v_000327 Label 3 Train/Test 1
Processing ... Video v_000328 Label 3 Train/Test 1
Processing ... Video v_000329 Label 3 Train/Test 1
Processing ... Video v_000330 Label 3 Train/Test 1
Processing ... Video v_000331 Label 3 Train/Test 1
Processing ... Video v_000332 Label 3 Train/Test 1
Processing ... Video v_000333 Label 3 Train/Test 1
Processing ... Video v_000334 Label 3 Train/Test 1
Processing ... Video v_000335 Label 3 Train/Test 1
Processing ... Video v_000336 Label 3 Train/Test 1
Processing ... Video v_000337 Label 3 Train/Test 1
Processing ... Video v_000338 Label 3 Train/Test 1
Processing ... Video v_000339 Label 3 Train/Test 1
Processing ... Video v_000340 Label 3 Train/Test 1
Processing ... Video v_000341 Label 3 Train/Test 1
Processing ... Video v_000342 Label 3 Train/Test 1
Processing ... Video v_000343 L

Processing ... Video v_000498 Label 4 Train/Test 1
Processing ... Video v_000499 Label 4 Train/Test 1
Processing ... Video v_000500 Label 4 Train/Test 1
Processing ... Video v_000501 Label 4 Train/Test 1
Processing ... Video v_000502 Label 4 Train/Test 1
Processing ... Video v_000503 Label 4 Train/Test 1
Processing ... Video v_000504 Label 4 Train/Test 1
Processing ... Video v_000505 Label 4 Train/Test 1
Processing ... Video v_000506 Label 4 Train/Test 1
Processing ... Video v_000507 Label 4 Train/Test 1
Processing ... Video v_000508 Label 4 Train/Test 1
Processing ... Video v_000509 Label 4 Train/Test 1
Processing ... Video v_000510 Label 4 Train/Test 1
Processing ... Video v_000511 Label 4 Train/Test 1
Processing ... Video v_000512 Label 4 Train/Test 1
Processing ... Video v_000513 Label 4 Train/Test 1
Processing ... Video v_000514 Label 4 Train/Test 1
Processing ... Video v_000515 Label 4 Train/Test 1
Processing ... Video v_000516 Label 4 Train/Test 1
Processing ... Video v_000517 L

Processing ... Video v_000678 Label 6 Train/Test 2
Processing ... Video v_000679 Label 6 Train/Test 2
Processing ... Video v_000680 Label 6 Train/Test 2
Processing ... Video v_000681 Label 6 Train/Test 2
Processing ... Video v_000682 Label 6 Train/Test 2
Processing ... Video v_000683 Label 6 Train/Test 2
Processing ... Video v_000684 Label 6 Train/Test 2
Processing ... Video v_000685 Label 6 Train/Test 2
Processing ... Video v_000686 Label 6 Train/Test 2
Processing ... Video v_000687 Label 6 Train/Test 2
Processing ... Video v_000688 Label 6 Train/Test 1
Processing ... Video v_000689 Label 6 Train/Test 1
Processing ... Video v_000690 Label 6 Train/Test 1
Processing ... Video v_000691 Label 6 Train/Test 1
Processing ... Video v_000692 Label 6 Train/Test 1
Processing ... Video v_000693 Label 6 Train/Test 1
Processing ... Video v_000694 Label 6 Train/Test 1
Processing ... Video v_000695 Label 6 Train/Test 1
Processing ... Video v_000696 Label 6 Train/Test 1
Processing ... Video v_000697 L

Processing ... Video v_000852 Label 7 Train/Test 1
Processing ... Video v_000853 Label 7 Train/Test 1
Processing ... Video v_000854 Label 7 Train/Test 1
Processing ... Video v_000855 Label 7 Train/Test 1
Processing ... Video v_000856 Label 7 Train/Test 1
Processing ... Video v_000857 Label 7 Train/Test 1
Processing ... Video v_000858 Label 7 Train/Test 1
Processing ... Video v_000859 Label 7 Train/Test 1
Processing ... Video v_000860 Label 7 Train/Test 1
Processing ... Video v_000861 Label 7 Train/Test 1
Processing ... Video v_000862 Label 7 Train/Test 1
Processing ... Video v_000863 Label 7 Train/Test 1
Processing ... Video v_000864 Label 7 Train/Test 1
Processing ... Video v_000865 Label 7 Train/Test 1
Processing ... Video v_000866 Label 7 Train/Test 1
Processing ... Video v_000867 Label 7 Train/Test 1
Processing ... Video v_000868 Label 7 Train/Test 1
Processing ... Video v_000869 Label 7 Train/Test 1
Processing ... Video v_000870 Label 7 Train/Test 1
Processing ... Video v_000871 L

Processing ... Video v_001015 Label 8 Train/Test 1
Processing ... Video v_001016 Label 8 Train/Test 1
Processing ... Video v_001017 Label 8 Train/Test 1
Processing ... Video v_001018 Label 8 Train/Test 1
Processing ... Video v_001019 Label 8 Train/Test 1
Processing ... Video v_001020 Label 8 Train/Test 1
Processing ... Video v_001021 Label 8 Train/Test 1
Processing ... Video v_001022 Label 8 Train/Test 1
Processing ... Video v_001023 Label 8 Train/Test 1
Processing ... Video v_001024 Label 8 Train/Test 1
Processing ... Video v_001025 Label 8 Train/Test 1
Processing ... Video v_001026 Label 8 Train/Test 1
Processing ... Video v_001027 Label 8 Train/Test 1
Processing ... Video v_001028 Label 8 Train/Test 1
Processing ... Video v_001029 Label 8 Train/Test 1
Processing ... Video v_001030 Label 8 Train/Test 1
Processing ... Video v_001031 Label 8 Train/Test 1
Processing ... Video v_001032 Label 8 Train/Test 1
Processing ... Video v_001033 Label 8 Train/Test 1
Processing ... Video v_001034 L

Processing ... Video v_001177 Label 9 Train/Test 1
Processing ... Video v_001178 Label 9 Train/Test 1
Processing ... Video v_001179 Label 9 Train/Test 1
Processing ... Video v_001180 Label 9 Train/Test 1
Processing ... Video v_001181 Label 9 Train/Test 1
Processing ... Video v_001182 Label 9 Train/Test 1
Processing ... Video v_001183 Label 9 Train/Test 1
Processing ... Video v_001184 Label 9 Train/Test 1
Processing ... Video v_001185 Label 9 Train/Test 1
Processing ... Video v_001186 Label 9 Train/Test 1
Processing ... Video v_001187 Label 9 Train/Test 1
Processing ... Video v_001188 Label 9 Train/Test 1
Processing ... Video v_001189 Label 9 Train/Test 1
Processing ... Video v_001190 Label 9 Train/Test 1
Processing ... Video v_001191 Label 9 Train/Test 1
Processing ... Video v_001192 Label 9 Train/Test 1
Processing ... Video v_001193 Label 9 Train/Test 1
Processing ... Video v_001194 Label 9 Train/Test 1
Processing ... Video v_001195 Label 9 Train/Test 1
Processing ... Video v_001196 L

Processing ... Video v_001484 Label 11 Train/Test 1
Processing ... Video v_001485 Label 11 Train/Test 1
Processing ... Video v_001486 Label 11 Train/Test 1
Processing ... Video v_001487 Label 11 Train/Test 1
Processing ... Video v_001488 Label 11 Train/Test 1
Processing ... Video v_001489 Label 11 Train/Test 1
Processing ... Video v_001490 Label 11 Train/Test 1
Processing ... Video v_001491 Label 11 Train/Test 1
Processing ... Video v_001492 Label 11 Train/Test 1
Processing ... Video v_001493 Label 11 Train/Test 1
Processing ... Video v_001494 Label 11 Train/Test 1
Processing ... Video v_001495 Label 11 Train/Test 1
Processing ... Video v_001496 Label 11 Train/Test 1
Processing ... Video v_001497 Label 11 Train/Test 1
Processing ... Video v_001498 Label 11 Train/Test 1
Processing ... Video v_001499 Label 11 Train/Test 1
Processing ... Video v_001500 Label 11 Train/Test 1
Processing ... Video v_001501 Label 11 Train/Test 1
Processing ... Video v_001502 Label 11 Train/Test 1
Processing .

Processing ... Video v_001797 Label 14 Train/Test 2
Processing ... Video v_001798 Label 14 Train/Test 2
Processing ... Video v_001799 Label 14 Train/Test 2
Processing ... Video v_001800 Label 14 Train/Test 2
Processing ... Video v_001801 Label 14 Train/Test 2
Processing ... Video v_001802 Label 14 Train/Test 2
Processing ... Video v_001803 Label 14 Train/Test 2
Processing ... Video v_001804 Label 14 Train/Test 2
Processing ... Video v_001805 Label 14 Train/Test 2
Processing ... Video v_001806 Label 14 Train/Test 2
Processing ... Video v_001807 Label 14 Train/Test 2
Processing ... Video v_001808 Label 14 Train/Test 2
Processing ... Video v_001809 Label 14 Train/Test 2
Processing ... Video v_001810 Label 14 Train/Test 2
Processing ... Video v_001811 Label 14 Train/Test 2
Processing ... Video v_001812 Label 14 Train/Test 2
Processing ... Video v_001813 Label 14 Train/Test 2
Processing ... Video v_001814 Label 14 Train/Test 2
Processing ... Video v_001815 Label 14 Train/Test 2
Processing .

Processing ... Video v_001956 Label 15 Train/Test 1
Processing ... Video v_001957 Label 15 Train/Test 1
Processing ... Video v_001958 Label 15 Train/Test 1
Processing ... Video v_001959 Label 15 Train/Test 1
Processing ... Video v_001960 Label 15 Train/Test 1
Processing ... Video v_001961 Label 15 Train/Test 1
Processing ... Video v_001962 Label 15 Train/Test 1
Processing ... Video v_001963 Label 15 Train/Test 1
Processing ... Video v_001964 Label 15 Train/Test 1
Processing ... Video v_001965 Label 15 Train/Test 1
Processing ... Video v_001966 Label 15 Train/Test 1
Processing ... Video v_001967 Label 15 Train/Test 1
Processing ... Video v_001968 Label 15 Train/Test 1
Processing ... Video v_001969 Label 15 Train/Test 1
Processing ... Video v_001970 Label 15 Train/Test 1
Processing ... Video v_001971 Label 15 Train/Test 1
Processing ... Video v_001972 Label 15 Train/Test 1
Processing ... Video v_001973 Label 15 Train/Test 1
Processing ... Video v_001974 Label 15 Train/Test 1
Processing .

In [6]:
def predict_on_test_svm(preds_svm):
    correct = 0.0
    total = 0.0
    for i in range(len(preds_svm)):
        if preds_svm[i] == test_lbls_svm[i]:
            correct += 1.0
        
        total += 1.0
       
    return (correct/total ) * 100

start_time = time.time()
preds_svm = model_svm.predict(np.array(test_features_svm))
#print(preds_svm.shape)
accuracy_svm = predict_on_test_svm(preds_svm)
print('Test accuracy for SVM is ',accuracy_svm)
end_time = time.time() - start_time 
print('------------------------------------------------------------')
print("time taken to PREDICT with SVM  in seconds {:.3f}".format(end_time))

Test accuracy for SVM is  94.02112676056338
------------------------------------------------------------
time taken to PREDICT with SVM  in seconds 127.918


###  ****************Q3. REPORT is available in PDF format in this current folder *******************


## Submission
---
**Runnable source code in ipynb file and a pdf report are required**.

The report should be of 3 to 4 pages describing what you have done and learned in this homework and report performance of your model. If you have tried multiple methods, please compare your results. If you are using any external code, please cite it in your report. Note that this homework is designed to help you explore and get familiar with the techniques. The final grading will be largely based on your prediction accuracy and the different methods you tried (different architectures and parameters).

Please indicate clearly in your report what model you have tried, what techniques you applied to improve the performance and report their accuracies. The report should be concise and include the highlights of your efforts.